# Check Data Quality of the Files

checking data quality of the files
e.g. if there are redundant files or empty files, etc.

Steps:
- importing Boto because files are on S3

In [11]:
#!pip3 install s3fs boto3 mock --use-feature=2020-resolver
# there is a bug in the feature resolver, that the installation runs forever, even though the installation succeeded.
# you may have to interrupt the kernel and then s3fs is isntalled properly

In [12]:
import boto3
import pandas as pd

In [13]:
def file_list(bucket_name):
    """A function that lists all files (objects) in a given bucket (argument bucket_name) and puts it in a list"""
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)
    return [i.key for i in my_bucket.objects.all()]

In [14]:
file_list('dsti-ml-nlp-class')

['Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (2).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (3).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (4).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT.csv',
 'Data/MR_TAPIYA_JOHN_BANK_STATEMENT-part2.csv',
 'Data/Mr_Tochukwu_statement-part2.csv',
 'Data/mr_gani_olalekan_statement_July_9.xlsx']

In [26]:
import boto3
import os

def download_dir(client, resource, dist, local='/tmp', bucket='your_bucket'):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=dist):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_dir(client, resource, subdir.get('Prefix'), local, bucket)
        for file in result.get('Contents', []):
            dest_pathname = os.path.join(local, file.get('Key'))
            if not os.path.exists(os.path.dirname(dest_pathname)):
                os.makedirs(os.path.dirname(dest_pathname))
            resource.meta.client.download_file(bucket, file.get('Key'), dest_pathname)
            
def _start():
    client = boto3.client('s3')
    resource = boto3.resource('s3')
    download_dir(client, resource, 'clientconf/', './tmp', bucket='dsti-ml-nlp-class')
    
    
_start()

In [23]:
s3 = boto3.resource('s3')
for file in file_list('dsti-ml-nlp-class'):
    file = file[5:]
    print(file)
    s3.Bucket("dsti-ml-nlp-class").download_file(file, file)

MR_IBRAHIM_ASHAFA_STATEMENT (1).csv


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [15]:
def create_s3_file_uri(bucket_name, file_name):
    """takes a bucket name and filename as argument and forms an s3 uri from it."""
    s3_file_uri = "s3://" + bucket_name + "/" + file_name
    return s3_file_uri

In [16]:
def file_to_pandas(s3_file_uri):
    """put a given file into a pandas dataframe. Can differentiate between .xlsx and .csv"""
    if (s3_file_uri.endswith(".csv")):
        data = pd.read_csv(s3_file_uri)
    elif (s3_file_uri.endswith(".xlsx")):
        data = pd.read_excel(s3_file_uri)
    else:
        print("This Function can handle only .xlsx or .csv files")
        data = None
    return data

In [17]:
create_s3_file_uri("dsti-ml-nlp-class", "Data/mr_gani_olalekan_statement_July_9.xlsx")

's3://dsti-ml-nlp-class/Data/mr_gani_olalekan_statement_July_9.xlsx'

In [18]:
s3_test_uri = create_s3_file_uri("dsti-ml-nlp-class", "Data/mr_gani_olalekan_statement_July_9.xlsx")

In [19]:
df = file_to_pandas(s3_test_uri)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
def clean_dataframe (df):
    """takes a Dataframe as input, selects the column as header, which has TXN DATE in first column
    select only the columns that are not nan"""
    
    #selecting only rows with txn data and after
    tmp_df = df[list(df[df.iloc[:,0]=="TXN DATE"].index)[0]:]
    
    #removing first two lines of the dataframe and taking the third column as headers
    df_with_headers = pd.DataFrame(tmp_df)
    df_with_headers.columns = tmp_df.iloc[0].values
    df_with_headers.drop(df_with_headers.index[0], inplace=True)
    
    #reducing into a new df that has no nan columns
    df_reduced = df_with_headers[[el for el in test_data.iloc[0].values if isinstance(el, str)]]
    return df_reduced

In [ ]:
clean_dataframe(df)